# JourneyPatternID is a combination of Variant and Direction

## Plan
## Merge on southbound

### from routes_all(headsigns).csv derrive SOUTHBOUND (LineID,JourneyPatternID,Headsign)
#### Step1:
Append County to headsigns
#### Step2:
Derrive Southbound - for a given LineID there are two headsigns, substract the corresponding latitude for each headsign from the other.

### From Timetable - Derive Southbound Column
#### Step4:
derive to_lat
#### Step5:
derive from_lat
#### Step6:
subtract to get southbound


In [10]:
import pandas as pd
import math
from datetime import datetime, timedelta
import os

import matplotlib.pyplot as plt
%matplotlib inline

### Constants

In [17]:
lines = []
for i in os.listdir("bus_data/clean_data_final/"):
    lines.append(i.strip(".csv"))

In [18]:
lines

['1',
 '102',
 '104',
 '11',
 '111',
 '114',
 '116',
 '118',
 '120',
 '122',
 '123',
 '13',
 '130',
 '14',
 '140',
 '142',
 '145',
 '14C',
 '15',
 '150',
 '151',
 '15A',
 '15B',
 '16',
 '161',
 '16C',
 '17',
 '17A',
 '18',
 '184',
 '185',
 '220',
 '236',
 '238',
 '239',
 '25',
 '25A',
 '25B',
 '25X',
 '26',
 '27',
 '270',
 '27A',
 '27B',
 '27X',
 '29A',
 '31',
 '31A',
 '31B',
 '32',
 '32X',
 '33',
 '33A',
 '33B',
 '33X',
 '37',
 '38',
 '38A',
 '38B',
 '39',
 '39A',
 '4',
 '40',
 '40B',
 '40D',
 '41',
 '41A',
 '41B',
 '41C',
 '41X',
 '42',
 '43',
 '44',
 '44B',
 '45A',
 '46A',
 '46E',
 '47',
 '49',
 '51D',
 '51X',
 '53',
 '54A',
 '56A',
 '59',
 '61',
 '63',
 '65',
 '65B',
 '66',
 '66A',
 '66B',
 '66X',
 '67',
 '67X',
 '68',
 '68A',
 '69',
 '69X',
 '7',
 '70',
 '747',
 '75',
 '76',
 '76A',
 '77A',
 '79',
 '79A',
 '7B',
 '7D',
 '8',
 '83',
 '83A',
 '84',
 '84A',
 '84X',
 '9']

In [4]:
lines =['102', '104', '111', '114', '116', '118', '11', '120', '122', '123',
       '130', '13', '140', '142', '145', '14', '150', '151', '15A', '15B',
       '15', '161', '16', '17A', '17', '184', '185', '18', '1', '220',
       '236', '238', '239', '25A', '25B', '25X', '25', '26', '270', '27A',
       '27B', '27X', '27', '29A', '31B', '31', '32', '32X', '33A', '33B',
       '33X', '33', '37', '38A', '38', '39A', '39', '40B', '40D', '40',
       '41B', '41C', '41X', '41', '42', '43', '44B', '44', '45A', '46A',
       '46E', '47', '49', '4', '51D', '51X', '53', '54A', '56A', '59',
       '61', '63', '65B', '65', '66A', '66X', '66', '67', '68', '69X',
       '69', '70', '747', '75', '76A', '76', '77A', '79', '79A', '7B',
       '7D', '7', '83', '84', '84A', '84X', '8', '9']

service_types = ["weekday","weekday","weekday","weekday","weekday","saturday","sunday"]


# Map Timetable to SOUTHBOUND

In [5]:
#Existing Mapping
timetable_mapping = pd.read_csv('bus_data/static_data/final_timetable.csv')
timetable_mapping = timetable_mapping.drop('Unnamed: 0', 1)

In [6]:
timetable_mapping.head()

,LineID,day,departure,JourneyPatternID
0,102,saturday,05:45,1001
1,102,saturday,06:45,1001
2,102,saturday,07:10,1001
3,102,saturday,07:20,1001
4,102,saturday,07:05,1001


In [7]:
timetable_mapping.LineID.unique()

array(['102', '104', '111', '114', '116', '118', '11', '120', '122', '123',
       '130', '13', '140', '142', '145', '14', '150', '151', '15A', '15B',
       '15', '161', '16', '17A', '17', '184', '185', '18', '1', '220',
       '236', '238', '239', '25A', '25B', '25X', '25', '26', '270', '27A',
       '27B', '27X', '27', '29A', '31B', '31', '32', '32X', '33A', '33B',
       '33X', '33', '37', '38A', '38', '39A', '39', '40B', '40D', '40',
       '41B', '41C', '41X', '41', '42', '43', '44B', '44', '45A', '46A',
       '46E', '47', '49', '4', '51D', '51X', '53', '54A', '56A', '59',
       '61', '63', '65B', '65', '66A', '66X', '66', '67', '68', '69X',
       '69', '70', '747', '75', '76A', '76', '77A', '79', '79A', '7B',
       '7D', '7', '83', '84', '84A', '84X', '8', '9'], dtype=object)

In [8]:
for i in lines:
    if i not in timetable_mapping.LineID.unique():
        print(i)

In [9]:
#to drop
for i in timetable_mapping.LineID.unique():
    if i not in lines:
        print(i)

In [8]:
timetable_mapping = timetable_mapping[~timetable_mapping['LineID'].isin(['32B','32A','33D','90'])]
timetable_mapping.dropna(axis=0, subset=['LineID'], inplace=True)

In [10]:
timetable_mapping.columns

Index(['fullname', 'LineID', 'southbound'], dtype='object')

In [11]:
timetable_mapping.to_csv('timetable_mapping.csv')

# MERGE TIMETABLES ON 'fullname' TO TIMETABLE MAPPING

# MAP HEADSIGNS TO SOUTHBOUND

In [95]:
heads = pd.read_csv('bus_data/static_data/heads.csv',index_col=0)

In [96]:
heads.head()

,LineID,JourneyPatternID,Headsign,Towards
0,1,1,Strand Road,"Strand Road, Dublin"
1,1,1001,Shanard Road,"Shanard Road, Dublin"
2,102,1,Airport,"Airport, Dublin"
3,102,1001,Sutton,"Sutton, Dublin"
4,104,1,Shanard Road,"Shanard Road, Dublin"


In [97]:
from geopy.geocoders import Nominatim

def get_lat(a):
    geolocator = Nominatim()
    location = Nominatim().geocode(a)
    return(location.latitude)



In [103]:
#INVESTIGATIONS
# num = 171
# heads.iloc[num]

# b = heads.iloc[num].values[-1]
# print(get_lat(b))

# a = list(heads.Towards)

# for i in range(167,len(a)):
#     print()
#     print(i)
#     print(get_lat(a[i]))

In [102]:
heads['Towards_Lat'] = heads['Towards'].apply(get_lat)

In [344]:
heads.head()

,LineID,JourneyPatternID,Headsign,Towards,Towards_Lat,From_Lat,southbound
0,1,1,Strand Road,"Strand Road, Dublin",53.333336,53.390694,True
1,1,1001,Shanard Road,"Shanard Road, Dublin",53.390694,53.333336,False
2,102,1,Airport,"Airport, Dublin",53.429736,53.390520,False
3,102,1001,Sutton,"Sutton, Dublin",53.390520,53.429736,True
4,104,1,Shanard Road,"Shanard Road, Dublin",53.390694,53.362922,False


In [116]:
pair_list = []
grouped = heads.groupby(['LineID'])
for key, df in grouped:
    pair_list.append(list(df.Towards_Lat))
    

In [117]:
reversed_list = []
for pair in pair_list:
    reversed_pair = reversed(pair)
    for r in reversed_pair:
        reversed_list.append(r)
        

In [118]:
heads['From_Lat'] = reversed_list

In [120]:
heads['southbound'] = (heads.Towards_Lat - heads.From_Lat)<0

In [123]:
head_map = heads[['LineID','JourneyPatternID','southbound']]

In [346]:
head_map.to_csv('bus_data/static_data/head_map.csv')

# Merge on southbound

In [127]:
timetable_mapping.shape
# timetable_mapping.head()

(557, 3)

In [128]:
head_map.shape
# head_map.head()

(227, 3)

In [130]:
timetable_mapping.head()

,fullname,LineID,southbound
0,102_saturday_From Dublin Airport Towards Sutto...,102,True
1,102_saturday_From Sutton Station Towards Dubli...,102,False
2,102_sunday_From Dublin Airport Towards Sutton ...,102,True
3,102_sunday_From Sutton Station Towards Dublin ...,102,False
4,102_weekday_From Dublin Airport Towards Sutton...,102,True


In [131]:
#Monday is 0, Sunday is 6
def get_direction(x):
    return x.split("_")[-1].strip('.csv')
    
timetable_mapping['direction'] = timetable_mapping['fullname'].apply(get_direction)

In [139]:
timetable_mapping.head()

,fullname,LineID,southbound,direction
0,102_saturday_From Dublin Airport Towards Sutto...,102,True,From Dublin Airport Towards Sutton Station
1,102_saturday_From Sutton Station Towards Dubli...,102,False,From Sutton Station Towards Dublin Airport
2,102_sunday_From Dublin Airport Towards Sutton ...,102,True,From Dublin Airport Towards Sutton Station
3,102_sunday_From Sutton Station Towards Dublin ...,102,False,From Sutton Station Towards Dublin Airport
4,102_weekday_From Dublin Airport Towards Sutton...,102,True,From Dublin Airport Towards Sutton Station


In [210]:
timetable_mapping.to_csv('bus_data/static_data/timetable_mapping.csv')

# Merge with Overall Timetable

In [326]:
timetable_mapping = pd.read_csv('bus_data/static_data/timetable_mapping.csv',index_col=0)

In [327]:
master_timetable = pd.read_csv('bus_data/static_data/timetable.csv',index_col=0)

In [328]:
master_timetable.direction.nunique()

221

In [329]:
timetable_mapping.direction.nunique()

197

In [330]:
master_timetable.line.unique()

array(['102', '104', '111', '114', '116', '118', '11', '120', '122', '123',
       '130', '13', '140', '142', '145', '14', '150', '151', '15a', '15b',
       '15n', '15', '161', '16', '17a', '17', '184', '185', '18', '1',
       '220', '236', '238', '239', '25a', '25b', '25n', '25x', '25', '26',
       '270', '27a', '27b', '27x', '27', '29a', '29n', '31b', '31n', '31',
       '32', '32a', '32x', '33a', '33b', '33d', '33n', '33x', '33', '37',
       '38a', '38', '39a', '39n', '39', '40b', '40d', '40', '41b', '41c',
       '41n', '41x', '41', '42n', '42', '43', '44b', '44', '45a', '46a',
       '46e', '46n', '47', '49n', '49', '4', '51d', '51x', '53', '54a',
       '56a', '59', '61', '63', '65b', '65', '66a', '66n', '66x', '66',
       '67n', '67', '68', '69x', '69', '70n', '70', '747', '75', '76a',
       '76', '77a', '77n', '77x', '79', '7b', '7d', '7n', '7', '83', '84',
       '84n', '84x', '88n', '8', '90', '9'], dtype=object)

In [331]:
master_timetable.columns

Index(['day', 'departure', 'direction', 'fullname', 'line'], dtype='object')

In [332]:
# backup_master = master_timetable.copy()

In [333]:
# master.to_csv("TIMETABLE.csv")

In [334]:
timetable_mapping.columns

Index(['fullname', 'LineID', 'southbound', 'direction'], dtype='object')

In [335]:
for i in timetable_mapping.fullname.unique():
    if i not in master_timetable.fullname.unique():
        print(i)

8_saturday_From Dalkey Towards Mountjoy Sq..csv
8_saturday_From Mountjoy Sq. Towards Dalkey.csv
8_sunday_From Dalkey Towards Mountjoy Sq..csv
8_sunday_From Mountjoy Sq. Towards Dalkey.csv


In [336]:
# replace these
for i in master_timetable.fullname.unique():
    if i not in timetable_mapping.fullname.unique():
        print(i)

15n_Friday Saturday_From D'Olier St. Towards Ellensborough.csv
25n_Friday Saturday_From Westmoreland Street Towards Adamstown.csv
29n_Friday Saturday_From D'Olier St. Towards Baldoyle Rd..csv
31n_Friday Saturday_From D'Olier St. Towards Howth.csv
32a_saturday_From Abbey St. Towards Malahide.csv
32a_saturday_From Malahide Towards Abbey St..csv
32a_sunday_From Abbey St. Towards Malahide.csv
32a_sunday_From Malahide Towards Abbey St..csv
32a_weekday_From Abbey St. Towards Malahide.csv
32a_weekday_From Malahide Towards Abbey St..csv
33d_weekday_From Custom House Quay Towards Portrane.csv
33d_weekday_From Portrane Towards Custom House Quay.csv
33n_Friday Saturday_From Westmoreland St. Towards Balbriggan.csv
39n_Friday Saturday_From Westmoreland St. Towards Tyrrelstown.csv
41n_Friday Saturday_From Westmoreland St. Towards Swords Manor.csv
42n_Friday Saturday_From D'Olier St. Towards Portmarnock.csv
46n_Friday Saturday_From D'Olier St. Towards Dundrum.csv
49n_Friday Saturday_From D'Olier St. 

# Merge

In [340]:
merged = pd.merge(timetable_mapping, master_timetable, on=['fullname'])

In [343]:
essential = merged[['LineID','day','departure','southbound']]
essential.head()

,LineID,day,departure,southbound
0,102,saturday,5:45,True
1,102,saturday,6:45,True
2,102,saturday,7:10,True
3,102,saturday,7:20,True
4,102,saturday,7:05,True


# Merge again
Merge essential with head_map on southbound

In [347]:
head_map.head()

,LineID,JourneyPatternID,southbound
0,1,1,True
1,1,1001,False
2,102,1,False
3,102,1001,True
4,104,1,False


In [348]:
final = pd.merge(essential,head_map,on=['LineID','southbound'])

In [349]:
final.head()

,LineID,day,departure,southbound,JourneyPatternID
0,102,saturday,5:45,True,1001
1,102,saturday,6:45,True,1001
2,102,saturday,7:10,True,1001
3,102,saturday,7:20,True,1001
4,102,saturday,7:05,True,1001


In [365]:
final_timetable = final[['LineID','day','departure','JourneyPatternID']]

In [366]:
final_timetable = final_timetable.dropna(axis=0, how='any')

In [369]:
def typo(x):
    if x == 'saturay':
        return 'saturday'
    elif x == 'sundday':
        return 'sunday'
    elif x == 'Friday Saturday':
        return 'saturday'
    else:
        return x

In [370]:
final_timetable['day'] = final_timetable['day'].apply(typo)

In [371]:
###TYPOS
final_timetable.day.value_counts()

weekday     7498
saturday    5891
sunday      3936
Name: day, dtype: int64

In [372]:
final_timetable.to_csv("bus_data/static_data/final_timetable.csv")

In [373]:
final_timetable.departure.value_counts()

17:00    227
23:30    227
16:00    226
12:00    211
13:00    211
18:00    209
11:00    209
15:00    207
17:30    207
14:00    204
10:00    203
19:00    202
20:00    195
15:30    192
13:30    190
9:00     189
9:30     187
11:30    186
21:00    185
16:30    182
14:30    181
18:30    180
20:30    179
10:30    178
22:00    173
21:30    171
12:30    170
8:30     169
22:30    167
19:30    167
        ... 
10:57      1
17:03      1
9:08       1
17:36      1
14:57      1
18:41      1
11:07      1
23:13      1
15:08      1
8:13       1
16:51      1
8:21       1
6:56       1
15:58      1
6:22       1
11:52      1
19:07      1
15:13      1
7:13       1
17:04      1
17:14      1
17:48      1
7:58       1
9:52       1
6:29       1
14:04      1
21:51      1
21:21      1
11:08      1
12:33      1
Name: departure, dtype: int64